In [1]:
import sys
from os.path import abspath,join
sys.path.append(abspath(join('..', 'module')))
from openTable import *
from AccesDB import *

import pandas as pd
import numpy as np
from statistics import mean

def strip_space(text):
    result = "".join(str(text).split())
    
    return result

In [2]:
data = pd.read_csv('../data/PTN_sbm_v2.csv') #WEB

data = data.replace({'Prodi_database': r'psgd'}, {'Prodi_database': 'pgsd'}, regex=True)
data = data.replace({'Prodi_database': r'tekink'}, {'Prodi_database': 'teknik'}, regex=True)
data = data.replace({'Prodi_database': r'sasatra '}, {'Prodi_database': 'sastra '}, regex=True)
data = data.replace({'Prodi_database': r'pend. '}, {'Prodi_database': 'pendidikan '}, regex=True)
data = data.replace({'Prodi_database': r'pendidika '}, {'Prodi_database': 'pendidikan '}, regex=True)
data = data.replace({'Prodi_database': r'pendidkan '}, {'Prodi_database': 'pendidikan '}, regex=True)
data = data.replace({'Prodi_database': r'pendikan '}, {'Prodi_database': 'pendidikan '}, regex=True)
data = data.replace({'Prodi_database': r'pendidikaan '}, {'Prodi_database': 'pendidikan '}, regex=True)
data = data.replace({'Prodi_database': r'majemen '}, {'Prodi_database': 'manajemen '}, regex=True)
data = data.replace({'Prodi_database': r'manajamen'}, {'Prodi_database': 'manajemen'}, regex=True)
data = data.replace({'Prodi_database': r'adminstrasi'}, {'Prodi_database': 'administrasi'}, regex=True)
data = data.replace({'Prodi_database': r'pancasilan'}, {'Prodi_database': 'pancasila'}, regex=True)
data = data.replace({'Prodi_database': r'pancasla '}, {'Prodi_database': 'pancasila '}, regex=True)
data = data.replace({'Prodi_database': r'kewarganegaan'}, {'Prodi_database': 'kewarganegaraan'}, regex=True)
data = data.replace({'Prodi_database': r'sd perairan'}, {'Prodi_database': 'sumber daya perikanan'}, regex=True)
data = data.replace({'Prodi_database': r'pembangnan'}, {'Prodi_database': 'pembangunan'}, regex=True)
data = data.replace({'Prodi_database': r'manajemen sumber perairan'}, {'Prodi_database': 'manajemen sumber daya perikanan'}, regex=True)
data = data.replace({'Prodi_database': r'kedokteran hewan'}, {'Prodi_database': 'dokter hewan'}, regex=True)
data = data.replace({'Prodi_database': r'bimbingan dan konseling'}, {'Prodi_database': 'bimbingan konseling'}, regex=True)
data = data.replace({'Prodi_database': r'kedokteran gigi'}, {'Prodi_database': 'pendidikan dokter gigi'}, regex=True)
data = data.replace({'Prodi_database': r'pendidikan jasmani, kesehatan dan rekreasi'}, {'Prodi_database': 'pendidikan jasmani, kesehatan, dan rekreasi'}, regex=True)
data = data.replace({'Prodi_database': r'pendidikan jasmani, kesehatan, rekreasi'}, {'Prodi_database': 'pendidikan jasmani, kesehatan, dan rekreasi'}, regex=True)
data = data.replace({'Prodi_database': r'pendidikan dasar sekolah dasar'}, {'Prodi_database': 'pendidikan guru sekolah dasar'}, regex=True)
data = data.replace({'Prodi_database': r'pendidikan guru paud'}, {'Prodi_database': 'pendidikan guru pendidikan anak usia dini (paud)'}, regex=True)
data = data.replace({'Prodi_database': r' dan komunikasi visual'}, {'Prodi_database': ' komunikasi visual'}, regex=True)
data = data.replace({'Prodi_database': r'teknik elektronika '}, {'Prodi_database': 'teknik elektronika'}, regex=True)
data = data.replace({'Prodi_database': r'pendidikan jasmani kesehatan dan rekreasi'}, {'Prodi_database': 'pendidikan jasmani, kesehatan, dan rekreasi'}, regex=True)
data = data.replace({'Prodi_database': r'pancasila kewarganegaraan'}, {'Prodi_database': 'pancasila dan kewarganegaraan'}, regex=True)
data = data.replace({'Prodi_database': r'pendidikan guru pendidikan anak usia dini'}, {'Prodi_database': 'pendidikan guru pendidikan anak usia dini (paud)'}, regex=True)

data.Universitas_database = data.Universitas_database.apply(strip_space)
data.Prodi_database = data.Prodi_database.apply(strip_space)
data['namagabungan'] = data.Prodi_database.str.cat(data.Universitas_database.astype(str), sep='')

sumber = pd.read_csv('../data/database_ptn.csv',encoding='latin1',index_col='no') #DB
sumber = sumber.drop_duplicates(subset=['name', 'departmentname'],keep=False)

merged = pd.merge(data,sumber, on='namagabungan',how='left')
merged = merged.sort_values(['Universitas_database','Prodi_database'])
merged = merged[
    ['universityId','facultydepartmentid','Universitas_database','Prodi_database','Jenis','accreditation',
     'DT 2020','P 2019','rataan','s_baku','max','min','namagabungan'
     ]
]

keketatan = []
for index, row in merged.iterrows():
    if row['P 2019'] != 0:
        ketat = (row['DT 2020']/row['P 2019'])*100
        keketatan.append(ketat)
    else:
        keketatan.append(0)

merged.facultydepartmentid.fillna(0, inplace=True)
merged.accreditation.fillna('D', inplace=True)
merged['persentase_keketatan'] = keketatan
merged['rataan'] = merged['rataan'].str.replace(',','.')
merged['s_baku'] = merged['s_baku'].str.replace(',','.')
merged['max'] = merged['max'].str.replace(',','.')
merged['min'] = merged['min'].str.replace(',','.')
merged['rataan'] = merged['rataan'].astype('float')
merged['s_baku'] = merged['s_baku'].astype('float')
merged['max'] = merged['max'].astype('float')
merged['min'] = merged['min'].astype('float')

rataan_baru = []
s_baku_baru = []

for index, row in merged.iterrows():
    if np.isnan(row['rataan']):
        try:
            jurusan = row['Prodi_database']
            akreditasi = row['accreditation']
            keketatan = row['persentase_keketatan']
            tabel = merged.loc[(merged['Prodi_database']==str(jurusan)) & (merged['accreditation']==str(akreditasi))]
            not_null = tabel[pd.notnull(tabel['rataan'])]
            mean_rataan = mean(not_null['rataan'].tolist())
            mean_s_baku = mean(not_null['s_baku'].tolist())
            mean_keketatan = mean(not_null['persentase_keketatan'].tolist())
            
            if keketatan > 100:
                rataan = float('NaN')
                s_baku = float('NaN')
            else:
                rataan = ((100-keketatan)/(100-mean_keketatan))*mean_rataan
                s_baku = ((100-keketatan)/(100-mean_keketatan))*mean_s_baku
                
#             print(jurusan,akreditasi,tabel.shape,rataan,s_baku)
            
        except Exception as e:
            rataan = float('NaN')
            s_baku = float('NaN')
#             print(e,'gak buat')
    else:
        rataan = row['rataan']
        s_baku = row['s_baku']
    
    rataan_baru.append(rataan)
    s_baku_baru.append(s_baku)
    
merged['rataan_baru'] = rataan_baru
merged['s_baku_baru'] = s_baku_baru

teknologidanmanajemenperikananbudidaya D (2, 14) 597.015498676684 13.28673849568855
mean requires at least one data point gak buat
mean requires at least one data point gak buat
mean requires at least one data point gak buat
mean requires at least one data point gak buat
mean requires at least one data point gak buat
mean requires at least one data point gak buat
mean requires at least one data point gak buat
desainkomunikasivisual D (3, 14) nan nan
mean requires at least one data point gak buat
mean requires at least one data point gak buat
mean requires at least one data point gak buat
mean requires at least one data point gak buat
mean requires at least one data point gak buat
desaininterior A (3, 14) 438.81898709847917 11.282939302998237
desainkomunikasivisual A (5, 14) 514.8051395820721 10.061812246157567
mean requires at least one data point gak buat
nan D (365, 14) 256.94368753980547 7.090454996996253
mean requires at least one data point gak buat
mean requires at least one data

mean requires at least one data point gak buat
pendidikanbahasainggris B (34, 14) nan nan
mean requires at least one data point gak buat
mean requires at least one data point gak buat
mean requires at least one data point gak buat
mean requires at least one data point gak buat
mean requires at least one data point gak buat
mean requires at least one data point gak buat
mean requires at least one data point gak buat
mean requires at least one data point gak buat
mean requires at least one data point gak buat
mean requires at least one data point gak buat
mean requires at least one data point gak buat
mean requires at least one data point gak buat
nan D (365, 14) 559.7205733975221 15.445693858321565
nan D (365, 14) 564.3185584725169 15.57257693750233
nan D (365, 14) 580.1517997609292 16.009501019533644
nan D (365, 14) 550.4385193634425 15.18955218370606
nan D (365, 14) 668.0535876034943 18.43518299219026
nan D (365, 14) 668.0535876034943 18.43518299219026
nan D (365, 14) 668.053587603494

biologi B (20, 14) 417.14062266500616 9.812328767123288
ekonomipembangunan B (14, 14) nan nan
mean requires at least one data point gak buat
mean requires at least one data point gak buat
ilmuhubunganinternasional B (7, 14) 394.99021272337194 9.066632084645017
ilmuhukum B (24, 14) nan nan
mean requires at least one data point gak buat
mean requires at least one data point gak buat
ilmukeperawatan B (8, 14) 530.8742261640799 13.904365410199558
mean requires at least one data point gak buat
kesehatanmasyarakat B (9, 14) 452.10276916076833 9.375655137822623
kimia B (26, 14) nan nan
manajemen B (25, 14) 143.32361193244307 3.4115241401044827
matematika B (22, 14) nan nan
nan D (365, 14) nan nan
nan D (365, 14) 413.55698280216313 11.41225613802254
nan D (365, 14) nan nan
nan D (365, 14) 381.74490720199674 10.534390281251577
nan D (365, 14) 668.0535876034943 18.43518299219026
nan D (365, 14) nan nan
nan D (365, 14) nan nan
nan D (365, 14) 272.169980134757 7.510630107929366
nan D (365, 14) nan

sastrajepang B (4, 14) 501.4023736984419 10.96597582133351
sastraperancis B (3, 14) 446.6141815880133 7.525345109995506
sosiologi A (17, 14) 603.5895201904857 14.163897453456903
statistika A (8, 14) 660.9432815544104 18.792711970106016
teknikarsitektur A (4, 14) 622.9234762474125 15.528290527903367
teknikelektro A (9, 14) 630.5496511758213 18.09724555531035
teknikgeologi A (4, 14) 663.6472825170398 19.895295694969626
teknikindustri A (10, 14) 654.4987674266569 18.527753074556507
teknikkelautan A (2, 14) 576.4471941391942 15.131052503052503
tekniklingkungan A (8, 14) 644.7681503851843 14.66924448492254
teknikmesin A (11, 14) 615.0191605361964 15.622646612686873
mean requires at least one data point gak buat
teknikperkapalan A (4, 14) 626.341371884696 15.697638441546465
teknikpertambangan B (5, 14) 619.9912727272726 15.50690909090909
tekniksipil A (16, 14) 658.4940406652034 18.188728176524155
tekniksistemperkapalan A (2, 14) 641.055327249163 18.127732476509344
akuntansi B (24, 14) 539.48

mean requires at least one data point gak buat
ilmupolitik C (4, 14) 490.60056019656014 12.54176904176904
mean requires at least one data point gak buat
mean requires at least one data point gak buat
keperawatan B (4, 14) 578.1627979651163 11.868728197674416
mean requires at least one data point gak buat
kimia B (26, 14) 598.176708436164 11.78878508601402
manajemen B (25, 14) 557.1876037221266 13.262706228482386
mean requires at least one data point gak buat
nan D (365, 14) nan nan
nan D (365, 14) 607.6266299308164 16.76767900294692
mean requires at least one data point gak buat
pendidikanbahasaarab B (3, 14) 392.41461460299024 9.61649005661199
pendidikanbahasadansastraindonesia B (20, 14) 513.1636249348661 13.402338420594758
pendidikanbahasainggris B (34, 14) 583.4478380427156 14.466917690922328
pendidikanbiologi B (23, 14) 560.8311269015434 12.860396642944355
pendidikanekonomi B (18, 14) 550.0970765781304 5.686443601241808
mean requires at least one data point gak buat
pendidikanguru

...

akuntansi B (24, 14) 603.9921186430973 17.467526133469182
mean requires at least one data point gak buat
mean requires at least one data point gak buat
biologi B (20, 14) 444.8081129438077 10.463146491473303
dokterhewan B (3, 14) 650.9028354391132 20.982587396325865
mean requires at least one data point gak buat
fisika B (26, 14) 375.2984885681413 8.799032990141873
ilmuhukum B (24, 14) 398.7901920061796 10.293142307316257
ilmukesehatanmasyarakat B (6, 14) 581.0717302811389 16.63147222384383
ilmukomputer B (7, 14) 625.4868102763348 18.562236548450898
ilmukomunikasi B (24, 14) 485.51632886428956 12.253146143672732
ilmupeternakan A (2, 14) 222.71575573362807 5.347222028665137
ilmupolitik B (13, 14) nan nan
mean requires at least one data point gak buat
kimia B (26, 14) 466.2678309015895 9.189142896237263
manajemen C (5, 14) 537.7644098343208 14.041579451520203
mean requires at least one data point gak buat
matematika B (22, 14) 484.6824354014277 14.807241065129109
nan D (365, 14) 483.9980

nan D (365, 14) 364.39286596554234 10.055554359376504
nan D (365, 14) 222.68452920116482 6.1450609973967545
nan D (365, 14) 580.4400023440196 16.0174540751817
mean requires at least one data point gak buat
pendidikanbahasadansastraindonesia B (20, 14) 482.06495074303325 12.590131678537267
pendidikanbahasainggris B (34, 14) 449.33026732506437 11.141397001724636
pendidikanbahasajepang B (7, 14) 483.7800898541485 12.104452516642818
pendidikanbiologi B (23, 14) 383.57400555776104 8.795720524017467
pendidikanekonomi B (18, 14) 254.1563496664115 2.6272558241277486
mean requires at least one data point gak buat
pendidikangeografi A (11, 14) 540.8081395447548 16.055781981551593
pendidikangurupendidikananakusiadini(paud)(paud) D (29, 14) 417.51586651053873 10.802107728337239
pendidikangurusekolahdasar(pgsd) A (12, 14) 537.5094626563631 15.690936853262103
pendidikanilmupengetahuanalam B (3, 14) 388.2177955911824 8.513891783567134
mean requires at least one data point gak buat
mean requires at le

teknikelektro B (29, 14) 435.5304102251891 10.57756475827934
teknikmesin B (23, 14) 344.86108808429174 8.28018619317759
tekniksipil A (16, 14) 610.9523622417942 16.87554595694692
teknologihasilperikanan A (8, 14) nan nan
agribisnis B (16, 14) 538.8678543220711 12.099271610355421
agroteknologi B (15, 14) 479.39390979848594 13.380761275189252
akuntansi C (7, 14) 427.27523943447864 13.278352082699909
mean requires at least one data point gak buat
mean requires at least one data point gak buat
mean requires at least one data point gak buat
ekonomipembangunan B (14, 14) 456.41433622324325 11.464220475920536
mean requires at least one data point gak buat
ilmuhukum B (24, 14) 427.4267217513043 11.032277526192694
mean requires at least one data point gak buat
mean requires at least one data point gak buat
mean requires at least one data point gak buat
pendidikanbahasainggris B (34, 14) 386.77875675636506 9.590397073648838
pendidikanbiologi B (23, 14) 506.85121493090867 11.62258539211581
mean r

mean requires at least one data point gak buat
mean requires at least one data point gak buat
mean requires at least one data point gak buat
mean requires at least one data point gak buat
teknikelektro B (29, 14) 588.248865758697 14.286580972221447
teknikindustri B (13, 14) 605.2035276594389 12.952221593932588
teknikkimia B (7, 14) 593.7541881747338 12.467254428562786
teknikmesin B (23, 14) 561.0303354727685 13.47045462142705
mean requires at least one data point gak buat
tekniksipil B (27, 14) 573.4342795925511 11.71158030207437
mean requires at least one data point gak buat
agribisnis B (16, 14) 646.729212510024 14.52109703287891
agroekoteknologi B (7, 14) 621.101927862812 17.073518836557174
antropologisosial A (3, 14) 635.9959396305993 13.744046608465656
arsitektur A (8, 14) 649.1557356975238 17.912853017193566
biologi A (29, 14) 613.0680006405489 15.516939492112481
mean requires at least one data point gak buat
fisika B (26, 14) 588.9020588863905 13.807059718981497
ilmuadministrasi

mean requires at least one data point gak buat
mean requires at least one data point gak buat
biologi A (29, 14) 514.8062305030392 13.029903894073545
ekonomiislam B (4, 14) 526.0950456498852 13.854974907863882
mean requires at least one data point gak buat
fisika B (26, 14) 162.75699759332656 3.8159071640921387
mean requires at least one data point gak buat
ilmuadministrasinegara B (8, 14) 567.8586863369762 13.233934727277925
mean requires at least one data point gak buat
ilmuhubunganinternasional B (7, 14) 574.3690733589375 13.184106596056504
ilmuhukum A (26, 14) 491.41098521586497 13.345779302352124
mean requires at least one data point gak buat
ilmukomunikasi B (24, 14) 585.5021782316962 14.776524147176566
ilmupemerintahan B (7, 14) 562.3994822345666 9.364584425112671
ilmupolitik B (13, 14) 481.8906326227902 10.612168944054135
mean requires at least one data point gak buat
ilmutanah B (7, 14) 370.9224824590712 7.793141404016781
mean requires at least one data point gak buat
mean req

In [3]:
len(merged.loc[merged.rataan_baru.notnull()])

2317

In [4]:
# merged.to_csv('../data/univ_sbm_with_rerata.csv',index=False)

In [5]:
# merged['facultydepartmentid'] = merged['facultydepartmentid'].astype(pd.Int64Dtype())
merged['universityId'] = merged['universityId'].astype(pd.Int64Dtype())
# merged.facultydepartmentid.fillna(0, inplace=True)

server,koneksi = Connection_2()

for i,j in merged.iterrows():
    status = to_db(koneksi,'rasionalisasi_kampus',
                   ['jurusanId','jenis','tahun_lalu_rerata','tahun_lalu_sbaku','tahun_lalu_kuota','tahun_lalu_peminat'],
                   [j.facultydepartmentid, j.Jenis, j.rataan_baru, j.s_baku_baru, j['DT 2020'], j['P 2019']])
    
    print(i,status)
    
koneksi.commit()
koneksi.close()
server.stop()

1169 berhasil
1170 berhasil
1207 berhasil
1171 berhasil
1172 berhasil
1173 berhasil
1205 berhasil
1177 berhasil
1191 berhasil
1174 berhasil
1194 berhasil
1203 berhasil
1175 berhasil
1193 berhasil
1204 berhasil
1176 berhasil
1178 berhasil
1180 berhasil
1192 berhasil
1181 berhasil
1195 berhasil
1197 berhasil
1202 berhasil
1201 berhasil
1187 berhasil
1196 berhasil
1179 berhasil
1200 berhasil
1182 berhasil
1183 berhasil
1189 berhasil
1190 berhasil
1199 berhasil
(1054, "Unknown column 'nan' in 'field list'")
1184 gagal
1198 berhasil
1206 berhasil
1185 berhasil
1186 berhasil
(1054, "Unknown column 'nan' in 'field list'")
1188 gagal
(1054, "Unknown column 'nan' in 'field list'")
1152 gagal
(1054, "Unknown column 'nan' in 'field list'")
1149 gagal
(1054, "Unknown column 'nan' in 'field list'")
1151 gagal
(1054, "Unknown column 'nan' in 'field list'")
1148 gagal
(1054, "Unknown column 'nan' in 'field list'")
1150 gagal
(1054, "Unknown column 'nan' in 'field list'")
145 gagal
(1054, "Unknown col

1893 berhasil
1897 berhasil
1918 berhasil
1907 berhasil
1922 berhasil
1917 berhasil
1927 berhasil
1929 berhasil
1921 berhasil
1928 berhasil
1902 berhasil
1891 berhasil
1900 berhasil
1899 berhasil
1898 berhasil
1925 berhasil
1895 berhasil
1901 berhasil
1908 berhasil
1909 berhasil
1910 berhasil
1912 berhasil
1913 berhasil
1914 berhasil
1915 berhasil
1916 berhasil
1932 berhasil
1934 berhasil
1892 berhasil
1919 berhasil
1905 berhasil
1920 berhasil
1906 berhasil
1903 berhasil
1904 berhasil
(1054, "Unknown column 'nan' in 'field list'")
1911 gagal
363 berhasil
399 berhasil
397 berhasil
370 berhasil
393 berhasil
380 berhasil
372 berhasil
401 berhasil
389 berhasil
406 berhasil
394 berhasil
366 berhasil
367 berhasil
407 berhasil
400 berhasil
395 berhasil
385 berhasil
369 berhasil
398 berhasil
371 berhasil
364 berhasil
386 berhasil
387 berhasil
388 berhasil
404 berhasil
408 berhasil
409 berhasil
365 berhasil
368 berhasil
(1054, "Unknown column 'nan' in 'field list'")
391 gagal
(1054, "Unknown co

1543 berhasil
1557 berhasil
1520 berhasil
1508 berhasil
1521 berhasil
1515 berhasil
1537 berhasil
1541 berhasil
1564 berhasil
1560 berhasil
1500 berhasil
1504 berhasil
1517 berhasil
1524 berhasil
1516 berhasil
1559 berhasil
1563 berhasil
1550 berhasil
1552 berhasil
1553 berhasil
1555 berhasil
1554 berhasil
1562 berhasil
1510 berhasil
1528 berhasil
1532 berhasil
1525 berhasil
1526 berhasil
1533 berhasil
1527 berhasil
1529 berhasil
1531 berhasil
1534 berhasil
1530 berhasil
1519 berhasil
1536 berhasil
1539 berhasil
1535 berhasil
(1054, "Unknown column 'nan' in 'field list'")
2931 gagal
2917 berhasil
2871 berhasil
(1054, "Unknown column 'nan' in 'field list'")
2894 gagal
2872 berhasil
2916 berhasil
2919 berhasil
(1054, "Unknown column 'nan' in 'field list'")
2932 gagal
2879 berhasil
2905 berhasil
2875 berhasil
2893 berhasil
2877 berhasil
2921 berhasil
(1054, "Unknown column 'nan' in 'field list'")
2933 gagal
(1054, "Unknown column 'nan' in 'field list'")
2910 gagal
2920 berhasil
(1054, "Un

266 berhasil
261 berhasil
268 berhasil
267 berhasil
(1054, "Unknown column 'nan' in 'field list'")
269 gagal
(1054, "Unknown column 'nan' in 'field list'")
263 gagal
262 berhasil
1163 berhasil
1159 berhasil
(1054, "Unknown column 'nan' in 'field list'")
1166 gagal
1154 berhasil
1155 berhasil
1161 berhasil
1168 berhasil
1156 berhasil
1162 berhasil
1153 berhasil
1164 berhasil
1165 berhasil
1167 berhasil
1160 berhasil
1158 berhasil
1157 berhasil
853 berhasil
867 berhasil
857 berhasil
868 berhasil
861 berhasil
859 berhasil
869 berhasil
870 berhasil
862 berhasil
871 berhasil
874 berhasil
860 berhasil
875 berhasil
858 berhasil
873 berhasil
856 berhasil
872 berhasil
863 berhasil
865 berhasil
(1054, "Unknown column 'nan' in 'field list'")
864 gagal
876 berhasil
878 berhasil
855 berhasil
877 berhasil
854 berhasil
(1054, "Unknown column 'nan' in 'field list'")
866 gagal
526 berhasil
482 berhasil
481 berhasil
513 berhasil
(1054, "Unknown column 'nan' in 'field list'")
523 gagal
495 berhasil
(1054

2207 berhasil
(1054, "Unknown column 'nan' in 'field list'")
2237 gagal
2261 berhasil
701 berhasil
702 berhasil
659 berhasil
(1054, "Unknown column 'nan' in 'field list'")
685 gagal
658 berhasil
699 berhasil
(1054, "Unknown column 'nan' in 'field list'")
680 gagal
(1054, "Unknown column 'nan' in 'field list'")
708 gagal
663 berhasil
693 berhasil
678 berhasil
698 berhasil
692 berhasil
669 berhasil
714 berhasil
694 berhasil
679 berhasil
700 berhasil
696 berhasil
(1054, "Unknown column 'nan' in 'field list'")
684 gagal
(1054, "Unknown column 'nan' in 'field list'")
666 gagal
662 berhasil
697 berhasil
668 berhasil
683 berhasil
686 berhasil
687 berhasil
688 berhasil
690 berhasil
691 berhasil
712 berhasil
715 berhasil
716 berhasil
(1054, "Unknown column 'nan' in 'field list'")
709 gagal
710 berhasil
675 berhasil
677 berhasil
706 berhasil
(1054, "Unknown column 'nan' in 'field list'")
673 gagal
705 berhasil
713 berhasil
711 berhasil
(1054, "Unknown column 'nan' in 'field list'")
707 gagal
674

...

(1054, "Unknown column 'nan' in 'field list'")
226 gagal
(1054, "Unknown column 'nan' in 'field list'")
215 gagal
(1054, "Unknown column 'nan' in 'field list'")
222 gagal
243 berhasil
244 berhasil
224 berhasil
225 berhasil
449 berhasil
431 berhasil
452 berhasil
(1054, "Unknown column 'nan' in 'field list'")
475 gagal
440 berhasil
442 berhasil
435 berhasil
413 berhasil
460 berhasil
474 berhasil
453 berhasil
415 berhasil
(1054, "Unknown column 'nan' in 'field list'")
473 gagal
458 berhasil
(1054, "Unknown column 'nan' in 'field list'")
424 gagal
417 berhasil
451 berhasil
411 berhasil
423 berhasil
426 berhasil
457 berhasil
459 berhasil
439 berhasil
441 berhasil
461 berhasil
412 berhasil
450 berhasil
414 berhasil
445 berhasil
438 berhasil
437 berhasil
(1054, "Unknown column 'nan' in 'field list'")
427 gagal
(1054, "Unknown column 'nan' in 'field list'")
456 gagal
(1054, "Unknown column 'nan' in 'field list'")
463 gagal
454 berhasil
416 berhasil
(1054, "Unknown column 'nan' in 'field list'"

(1054, "Unknown column 'nan' in 'field list'")
1578 gagal
(1054, "Unknown column 'nan' in 'field list'")
1571 gagal
(1054, "Unknown column 'nan' in 'field list'")
1570 gagal
1573 berhasil
1624 berhasil
1617 berhasil
1618 berhasil
1582 berhasil
1586 berhasil
1588 berhasil
1589 berhasil
1592 berhasil
2529 berhasil
(1054, "Unknown column 'nan' in 'field list'")
2530 gagal
2500 berhasil
2499 berhasil
2536 berhasil
(1054, "Unknown column 'nan' in 'field list'")
2511 gagal
(1054, "Unknown column 'nan' in 'field list'")
2524 gagal
2504 berhasil
2515 berhasil
(1054, "Unknown column 'nan' in 'field list'")
2535 gagal
2505 berhasil
2528 berhasil
2502 berhasil
2510 berhasil
2532 berhasil
2501 berhasil
(1054, "Unknown column 'nan' in 'field list'")
2533 gagal
(1054, "Unknown column 'nan' in 'field list'")
2516 gagal
2506 berhasil
2537 berhasil
(1054, "Unknown column 'nan' in 'field list'")
2517 gagal
2503 berhasil
2514 berhasil
2518 berhasil
2538 berhasil
2519 berhasil
2493 berhasil
2513 berhasil


(1054, "Unknown column 'nan' in 'field list'")
2418 gagal
2442 berhasil
(1054, "Unknown column 'nan' in 'field list'")
2412 gagal
(1054, "Unknown column 'nan' in 'field list'")
2421 gagal
(1054, "Unknown column 'nan' in 'field list'")
2416 gagal
2434 berhasil
(1054, "Unknown column 'nan' in 'field list'")
2417 gagal
2419 berhasil
2422 berhasil
2436 berhasil
2443 berhasil
(1054, "Unknown column 'nan' in 'field list'")
2440 gagal
2427 berhasil
2428 berhasil
2439 berhasil
2408 berhasil
2424 berhasil
(1054, "Unknown column 'nan' in 'field list'")
2410 gagal
2426 berhasil
2438 berhasil
2437 berhasil
2415 berhasil
(1054, "Unknown column 'nan' in 'field list'")
2433 gagal
(1054, "Unknown column 'nan' in 'field list'")
2430 gagal
2409 berhasil
2407 berhasil
2423 berhasil
2425 berhasil
(1054, "Unknown column 'nan' in 'field list'")
2429 gagal
(1054, "Unknown column 'nan' in 'field list'")
2441 gagal
(1054, "Unknown column 'nan' in 'field list'")
2413 gagal
2411 berhasil
(1054, "Unknown column '

1212 berhasil
(1054, "Unknown column 'nan' in 'field list'")
964 gagal
958 berhasil
972 berhasil
965 berhasil
966 berhasil
970 berhasil
(1054, "Unknown column 'nan' in 'field list'")
968 gagal
978 berhasil
977 berhasil
971 berhasil
979 berhasil
(1054, "Unknown column 'nan' in 'field list'")
976 gagal
974 berhasil
(1054, "Unknown column 'nan' in 'field list'")
975 gagal
(1054, "Unknown column 'nan' in 'field list'")
973 gagal
(1054, "Unknown column 'nan' in 'field list'")
963 gagal
(1054, "Unknown column 'nan' in 'field list'")
969 gagal
(1054, "Unknown column 'nan' in 'field list'")
961 gagal
959 berhasil
962 berhasil
(1054, "Unknown column 'nan' in 'field list'")
967 gagal
(1054, "Unknown column 'nan' in 'field list'")
960 gagal
584 berhasil
601 berhasil
607 berhasil
614 berhasil
(1054, "Unknown column 'nan' in 'field list'")
623 gagal
583 berhasil
594 berhasil
611 berhasil
603 berhasil
581 berhasil
615 berhasil
(1054, "Unknown column 'nan' in 'field list'")
609 gagal
(1054, "Unknown 

2112 berhasil
2108 berhasil
(1054, "Unknown column 'nan' in 'field list'")
2117 gagal
2133 berhasil
(1054, "Unknown column 'nan' in 'field list'")
2120 gagal
2158 berhasil
2129 berhasil
(1054, "Unknown column 'nan' in 'field list'")
2118 gagal
2149 berhasil
2150 berhasil
2142 berhasil
(1054, "Unknown column 'nan' in 'field list'")
2134 gagal
2127 berhasil
(1054, "Unknown column 'nan' in 'field list'")
2100 gagal
(1054, "Unknown column 'nan' in 'field list'")
2115 gagal
(1054, "Unknown column 'nan' in 'field list'")
2116 gagal
2109 berhasil
2131 berhasil
(1054, "Unknown column 'nan' in 'field list'")
2151 gagal
2107 berhasil
2145 berhasil
2148 berhasil
2136 berhasil
2135 berhasil
(1054, "Unknown column 'nan' in 'field list'")
2144 gagal
2104 berhasil
2111 berhasil
2137 berhasil
(1054, "Unknown column 'nan' in 'field list'")
2105 gagal
2155 berhasil
2139 berhasil
(1054, "Unknown column 'nan' in 'field list'")
2146 gagal
(1054, "Unknown column 'nan' in 'field list'")
2157 gagal
(1054, "Un